In [1]:
# check how many missing values there are

import pandas as pd

df = pd.read_csv("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/data/demographics_data.csv")

In [4]:
# demographics: age
column_to_check = "Age"
invalid_entries = [" "]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")


Total invalid entries in column 'Age': 0
Row indices: []


In [5]:
# demographics: gender
column_to_check = "Gender"
invalid_entries = [" ", "other", "no_answer"]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")


Total invalid entries in column 'Gender': 7
Row indices: [70, 324, 492, 620, 772, 779, 964]


In [6]:
# demographics: education
column_to_check = "Education"
invalid_entries = [" ", "other", "no_answer"]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")


Total invalid entries in column 'Education': 4
Row indices: [72, 314, 428, 492]


In [7]:
# demographics: language
column_to_check = "Language"
invalid_entries = [" ", "english_other", "no_answer"]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")


Total invalid entries in column 'Language': 13
Row indices: [49, 53, 55, 172, 247, 429, 662, 825, 909, 947, 948, 950, 953]


In [8]:
# demographics: country
column_to_check = "Country"
invalid_entries = [" ", "other", "no_answer"]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")


Total invalid entries in column 'Country': 1
Row indices: [72]


In [ ]:
# demographics: SES
column_to_check = "Socioeconomic"
invalid_entries = [" ", "no_answer"]

# convert to string in case of mixed types
col_data = df[column_to_check].astype(str)

# create mask for invalid string entries
invalid_mask = col_data.isin(invalid_entries)

# create mask for actual NaNs
nan_mask = df[column_to_check].isna()

# combine both
mask = invalid_mask | nan_mask

# show results
print(f"Total invalid entries in column '{column_to_check}': {mask.sum()}")
print(f"Row indices: {df[mask].index.tolist()}")

In [ ]:
# total affected subjects

# define columns and invalid entries
columns_to_check = ["Age", "Gender", "Education", "Language", "Country", "Socioeconomic"]
invalid_entries = [" ", "no_answer", "other", "english_other"]

# convert specified columns to string
col_data = df[columns_to_check].astype(str)

# create masks for invalid values and NaNs
invalid_mask = col_data.isin(invalid_entries)
nan_mask = df[columns_to_check].isna()
combined_mask = invalid_mask | nan_mask

# find rows where any of the specified columns have invalid entries
rows_with_invalid = combined_mask.any(axis=1)

# get affected rows and unique Subject_IDs
affected_rows = df[rows_with_invalid]
affected_subjects = affected_rows["Subject_ID"].unique().tolist()

# output
print(f"Total affected subjects: {len(affected_subjects)}")
print(f"Affected Subject_IDs: {affected_subjects}")

In [14]:
### check scores and features

# load data
df_scores = pd.read_csv("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/data/language_scores_all_subjects.csv")
df_features = pd.read_csv("/Users/gilanorup/Desktop/Studium/MSc/MA/code/masters_thesis_gn/results/features/cookieTheft.csv")

# function to check for missing or zero entries
def check_missing_or_zero(df, id_column="Subject_ID", name=""):
    # create mask for NaN or 0.0 (excluding ID column from check)
    data_cols = df.drop(columns=[id_column])  # only numeric/feature columns
    missing_or_zero_mask = data_cols.isna() # | (data_cols == 0.0) -> that would show all subjects for cookie theft because of certain features

    # find affected rows
    affected_rows = missing_or_zero_mask.any(axis=1)
    affected_subjects = df.loc[affected_rows, id_column].tolist()

    print(f"{name}:")
    print(f"Total affected subjects: {len(affected_subjects)}")
    print(f"Affected Subject_IDs: {affected_subjects}\n")

    return affected_subjects

# run check
affected_scores = check_missing_or_zero(df_scores, name="Language Scores")
affected_features = check_missing_or_zero(df_features, name="Cookie Theft Features")

Language Scores:
Total affected subjects: 14
Affected Subject_IDs: [41, 43, 44, 46, 49, 50, 54, 56, 59, 61, 99, 253, 303, 1079]

Cookie Theft Features:
Total affected subjects: 44
Affected Subject_IDs: [56, 138, 141, 143, 149, 157, 163, 190, 198, 381, 385, 389, 469, 492, 500, 516, 585, 623, 626, 648, 724, 871, 893, 921, 938, 944, 957, 1009, 1065, 1112, 1125, 1130, 1155, 1169, 1222, 1223, 1257, 1258, 1276, 1298, 1308, 1315, 1318, 1361]



Total affected subjects: 27
Affected Subject_IDs: [138, 142, 145, 169, 172, 278, 303, 400, 488, 498, 631, 632, 707, 867, 922, 1059, 1069, 1125, 1128, 1133, 1244, 1297, 1298, 1302, 1308, 1319, 1349]
